# Set up the enviroment

In [1]:
%%capture --no-stderr
%pip install "pyautogen>=0.2.3"
%pip install -q kserve

In [2]:
# Imports
import os
import autogen
from kserve import KServeClient
from kserve import utils

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Parameters and Config

In [3]:
# Kubeflow
MODEL_NAME = "llama2" # specify the model name in kserve
namespace = utils.get_default_target_namespace() # automatically retrieve your KfaaS Namespace

# enviroment variables
os.environ["FORCED_API_KEY"] = "null" # force an API key for OpenAI API
os.environ["AUTOGEN_USE_DOCKER"] = "0" # don't launch agents in docker containers

# Define your model

In [4]:
import json
# initalise the kserve client
KServe = KServeClient()

print(KServe.get(MODEL_NAME, namespace=namespace, watch=True, timeout_seconds=120))

# get the "Llama2 model from KServe"
isvc_resp = KServe.get(MODEL_NAME, namespace=namespace)

# get the model internal URL
isvc_url = isvc_resp["status"]["address"]["url"]

# create the OpenAI API endpoint for AutoGen
isvc_url = isvc_url.split("/v1/models/")[0] + "/v1"

print(f"Kserve Endpoint: {isvc_url}")

NAME    READY      PREV    LATEST  URL
llama2  True          0       100  http://llama2.my-profile.svc.cluster.local
None
Kserve Endpoint: http://llama2.my-profile.svc.cluster.local/v1


In [ ]:
# load the config list from enviroment variables
config_list = autogen.config_list_from_dotenv(
    dotenv_file_path='.env',
    model_api_key_map={
        MODEL_NAME: {
            "api_key_env_var": "FORCED_API_KEY",
            "base_url": isvc_url
        }
    },
    filter_dict={
        "model": {
           MODEL_NAME,
        }
    }
)

config_list

INFO:root:Models available: ['llama2']


[{'api_key': 'null',
  'base_url': 'http://llama2.my-profile.svc.cluster.local/v1',
  'model': 'llama2'}]

# Configure your agents


In [6]:
# gpt_config = {
#     "cache_seed": 42,  # change the cache_seed for different trials
#     "temperature": 0, # set the forced prompt temp to 0
#     "config_list": config_list, # load the config from above
#     "timeout": 120, # set timeout to 120 seconds
# }

# user_proxy = autogen.UserProxyAgent(
#    name="Admin",
#    system_message="A human admin. Interact with the planner to discuss the plan. Plan execution needs to be approved by this admin.",
#    code_execution_config=False,
# )
# engineer = autogen.AssistantAgent(
#     name="Engineer",
#     llm_config=gpt_config,
#     system_message='''Engineer. You follow an approved plan. You write python/shell code to solve tasks. Wrap the code in a code block that specifies the script type. The user can't modify your code. So do not suggest incomplete code which requires others to modify. Don't use a code block if it's not intended to be executed by the executor.
# Don't include multiple code blocks in one response. Do not ask others to copy and paste the result. Check the execution result returned by the executor.
# If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
# ''',
# )
# scientist = autogen.AssistantAgent(
#     name="Scientist",
#     llm_config=gpt_config,
#     system_message="""Scientist. You follow an approved plan. You are able to categorize papers after seeing their abstracts printed. You don't write code."""
# )
# planner = autogen.AssistantAgent(
#     name="Planner",
#     system_message='''Planner. Suggest a plan. Revise the plan based on feedback from admin and critic, until admin approval.
# The plan may involve an engineer who can write code and a scientist who doesn't write code.
# Explain the plan first. Be clear which step is performed by an engineer, and which step is performed by a scientist.
# ''',
#     llm_config=gpt_config,
# )
# executor = autogen.UserProxyAgent(
#     name="Executor",
#     system_message="Executor. Execute the code written by the engineer and report the result.",
#     human_input_mode="NEVER",
#     code_execution_config={"last_n_messages": 3, "work_dir": "paper"},
# )
# critic = autogen.AssistantAgent(
#     name="Critic",
#     system_message="Critic. Double check plan, claims, code from other agents and provide feedback. Check whether the plan includes adding verifiable info such as source URL.",
#     llm_config=gpt_config,
# )

# groupchat = autogen.GroupChat(agents=[user_proxy, engineer, scientist, planner, executor, critic], messages=[], max_round=50)
# manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=gpt_config)

# Begin Group Chat


In [7]:
# user_proxy.initiate_chat(
#     manager,
#     message="""
# find papers on LLM applications from arxiv in the last week, create a markdown table of different domains.
# """,
# )

In [9]:
# 1. create an AssistantAgent instance named "assistant"
from autogen.agentchat.contrib.math_user_proxy_agent import MathUserProxyAgent
assistant = autogen.AssistantAgent(
    name="assistant",
    system_message="You are a helpful assistant.",
    llm_config={
        "timeout": 1600,
        "seed": 42,
        "config_list": config_list,
    },
)

# 2. create the MathUserProxyAgent instance named "mathproxyagent"
# By default, the human_input_mode is "NEVER", which means the agent will not ask for human input.
mathproxyagent = MathUserProxyAgent(
    name="mathproxyagent",
    human_input_mode="NEVER",
    code_execution_config={"use_docker": False},
)

math_problem = (
    "Find all $x$ that satisfy the inequality $(2x+10)(x+3)<(3x+9)(x+8)$. Express your answer in interval notation."
)

mathproxyagent.initiate_chat(assistant, problem=math_problem)

mathproxyagent (to assistant):

Let's use Python to solve a math problem.

Query requirements:
You should always use the 'print' function for the output and use fractions/radical forms instead of decimals.
You can use packages like sympy to help you.
You must follow the formats below to write your code:
```python
# your code
```

First state the key idea to solve the problem. You may choose from three ways to solve the problem:
Case 1: If the problem can be solved with Python code directly, please write a program to solve it. You can enumerate all possible arrangements if needed.
Case 2: If the problem is mostly reasoning, you can solve it by yourself directly.
Case 3: If the problem cannot be handled in the above two ways, please follow this process:
1. Solve the problem step by step (do not over-divide the steps).
2. Take out any queries that can be asked through Python (for example, any calculations or equations that can be calculated).
3. Wait for me to give the results.
4. Continu